## DAY1 Automatic Transfer (.ps1)

This should be kept and run in the Day 3 folder where the scene viewing eyetracking files are backed (Currently fishingowl). It will send new files (within 24hours) to Compute Canada. 

*Remember to replace with actual path*

In [ ]:
$currentDate = Get-Date
$localFolders = Get-ChildItem -Path . -Directory

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/FinalSceneViewingEyetracking/"

foreach ($localFolder in $localFolders) {

    $recentDirectories = Get-ChildItem -Path $localFolder.FullName -Directory | Where-Object {
        ($currentDate - $_.CreationTime).TotalHours -le 24
    }

    foreach ($dir in $recentDirectories) {

        $remotePath = "$remotePathBase/$($localFolder.Name)/$($dir.Name)"

        scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            
            $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
            Write-Output ("Directory '{0}' transferred to '{1}'. Size: {2:N2} MB" -f $dir.Name, $remotePath, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}
